# The Algorithm

In [33]:
import sympy
import random
import math

Step 1: get $h(x,y)$ from $f(x,y)$ and $g(x,y)$, remembering that $h(x,y) = x(g\partial_xf - f\partial_xg) - y(g\partial_yf - g\partial_yg)$

In [4]:
x = sympy.Symbol('x')
y = sympy.Symbol('y')
def hgetter(f,g):
    return x*(g*sympy.diff(f, x) - f*sympy.diff(g, x)) + y*(g*sympy.diff(f, y) - f*sympy.diff(g, y))

In [5]:
f = sympy.Poly('x**2 + 2*x*y + y**2')
g = sympy.Poly('x**3 + y')

In [6]:
h = hgetter(f,g)
print(h)

x*Poly(-x**4 - 4*x**3*y - 3*x**2*y**2 + 2*x*y + 2*y**2, x, y, domain='ZZ') + y*Poly(2*x**4 + 2*x**3*y - x**2 + y**2, x, y, domain='ZZ')


In [24]:
def monicMaker(h):
    '''
    This function peforms a rotation on a polynomial of two variables
    h(x,y). That is, it maps h(x,y) to h(x+ny, -nx + y)) for some n
    such that the resulting polynomial is monic in y.
    '''
    new_h = h
    while sympy.LC(new_h, y) not in sympy.CC:
        n = random.randint(1,10)
        new_h = h.as_expr().subs([(x, x + n*y), (y, -n*x + y)])
    new_h = new_h * (1/sympy.LC(new_h, y))
    return new_h.as_poly()

In [25]:
h = sympy.Poly(x**2*y**2 + 2*x*y + 4, x, y)
print(monicMaker(h))
print(sympy.LC(monicMaker(h), y) == 1)

Poly(576*x**4 - 2352/5*x**3*y + 3601/25*x**2*y**2 + 48/5*x**2 - 98/5*x*y**3 - 98/25*x*y + y**4 + 2/5*y**2 + 4/25, x, y, domain='QQ')
True


In [67]:
def newtonAutomorphism(h, q, p):
    '''
    This function performs the Newton Automorphism (x maps to x ** q
    and y maps to y(x ** p)) to a polynomial h, returning a sympy expr
    (not a polynomial). Here, p and q are expected to be rational
    numbers.
    '''
    return h.as_expr().subs([(x, x ** q), (y, y*x**p)])

In [38]:
newtonAutomorphism(h, 1/2, 1)
print(h)

Poly(x**2*y**2 + 2*x*y + 4, x, y, domain='ZZ')


In [63]:
def componentsOfy(F):
    '''
    This function returns a list with the homogenous
    components of h as a polynomial in y in increasing order.
    '''
    x = sympy.Symbol('x')
    y = sympy.Symbol('y')
    list_of_Fprimes = [F]
    list_of_fs = [F.eval(y, 0)]
    for j in range(1, sympy.degree(F, y)+1):
        nextFprime = sympy.diff(list_of_Fprimes[j-1], y)
        list_of_Fprimes.append(nextFprime)
        list_of_fs.append((1/math.factorial(j))*nextFprime.eval(y, 0).as_poly(x))
    return list_of_fs

def phiAutomorphism(h):
    '''
    This function performs the phi automorphism to a polynomial that's monic in y,
    so that the term that's with degree(h, y) - 1 banishes.
    '''
    x = sympy.Symbol('x')
    y = sympy.Symbol('y')
    list_of_bs = componentsOfy(h)
    b1 = list_of_bs[-2]
    return h.as_expr().subs([(y, y-b1/sympy.degree(h, y))]).as_poly(x,y)

In [66]:
monic_h = monicMaker(h)
# print(componentsOfy(monic_h))
print(monic_h)
phiAutomorphism(monic_h)

Poly(3969*x**4 - 8001/4*x**3*y + 24193/64*x**2*y**2 + 63/4*x**2 - 127/4*x*y**3 - 127/32*x*y + y**4 + 1/4*y**2 + 1/16, x, y, domain='QQ')


Poly(1.52587890625e-5*x**4 - 0.0078125*x**2*y**2 - 0.0009765625*x**2 + 1.0*y**4 + 0.25*y**2 + 0.0625, x, y, domain='RR')